In [1]:
import sys

sys.path.append('..')

In [2]:
from VisionSystem import VisionSystem, VisualObject, VideoStream
from VisionSystem.DetectionModel import ThreshBlob

def load_or_create_new_threshblob(path):
    try:
        model = ThreshBlob.load(path)
        print("Loaded " + path)
    except Exception:
        model = ThreshBlob()
    finally:
        return model

model_names = ["ball", "obstacle", "yellow_goal", "blue_goal"]
detection_models = {
    model_name: load_or_create_new_threshblob("../models/" + model_name + ".threshblob.pkl") \
                    for model_name in model_names
}

objects_to_track = {
    name: VisualObject(real_size=(0.043, 0.043, 0.043), detection_model=model, result_limit=1)
            for name, model in detection_models.items()
}

stream = VideoStream(downsample_scale=4)
vision_system = VisionSystem(objects_to_track, camera_pixel_width=stream.resolution[0])

Loaded ../models/ball.threshblob.pkl
Loaded ../models/obstacle.threshblob.pkl
Loaded ../models/yellow_goal.threshblob.pkl
Loaded ../models/blue_goal.threshblob.pkl


In [3]:
from DisplayPane import DisplayPane
from DisplayPane.Interactor import VisionSystemTuner

display = DisplayPane(
    video_stream=stream,
    vision_system=vision_system,
    interactors=[VisionSystemTuner(vision_system)]
)
display

DisplayPane(children=(HBox(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'r…

In [4]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for model_name in model_names:
    detection_models[model_name].save("../models/" + model_name + ".threshblob.pkl")

In [5]:
print(vision_system.objects_to_track)

{'ball': <VisionSystem.VisualObject.VisualObject object at 0x7f5c703ab8d0>, 'obstacle': <VisionSystem.VisualObject.VisualObject object at 0x7f5c703f6630>, 'yellow_goal': <VisionSystem.VisualObject.VisualObject object at 0x7f5c370c6f28>, 'blue_goal': <VisionSystem.VisualObject.VisualObject object at 0x7f5c370d36d8>}
